# Лабораторная работа №1: Сегментация и аннотация текста

## Этап 1: Импорт библиотек и загрузка данных


In [ ]:
import re
import csv
import os
from pathlib import Path
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4')

print("Библиотеки загружены успешно")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


Библиотеки загружены успешно


## Этап 2: Реализация сегментации и токенизации


In [ ]:
class AdvancedTokenizer:
    """Токенизатор с обработкой специальных случаев"""
    
    def __init__(self):
        self.patterns = {
            'url': r'https?://[^\s<>"{}|\\^`\[\]]+|www\.[^\s<>"{}|\\^`\[\]]+',
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'phone': r'(?:\+?\d{1,3}[-.\s]?)?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}|\d{1,3}[-.\s]?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}',
            'currency': r'[$\u20AC\u00A3\u00A5\u00A2]\s?\d{1,3}(?:[,\s]\d{3})*(?:\.\d{2})?',
            'percent': r'\d+(?:\.\d+)?%',
            'number': r'\b\d{1,3}(?:[,\s]\d{3})*(?:\.\d+)?\b',
            'math': r'\b[a-zA-Z]\s*[=<>]+\s*[a-zA-Z0-9+\-*/^()\s]+\b',
            'emoticon': r'[:;=][-]?[)(DPpOo]|\([-]?[:;=]|[)(DPpOo][-]?[:;=]',
            'abbreviation': r'\b(?:Dr|Mr|Mrs|Ms|Prof|Sr|Jr|Inc|Ltd|Corp|Co|St|Ave|Blvd|Rd|U\.S\.A|U\.K|etc|vs|e\.g|i\.e)\.',
            'address': r'\b(?:г\.|ул\.|д\.|кв\.|пр\.|пер\.|просп\.|бул\.)\s*[А-Яа-яA-Za-z0-9\s,.-]+',
        }
        
    def segment_sentences(self, text):
        """Сегментация текста на предложения"""
        sentences = sent_tokenize(text)
        return sentences
    
    def tokenize(self, text):
        """Токенизация текста с обработкой специальных случаев"""
        tokens = []
        
        # Сначала извлекаем все специальные паттерны
        special_tokens = []
        
        for pattern_name, pattern in self.patterns.items():
            for match in re.finditer(pattern, text, re.IGNORECASE):
                special_tokens.append((match.start(), match.end(), match.group(), pattern_name))
        
        # Сортируем по позиции
        special_tokens.sort(key=lambda x: x[0])
        
        # Удаляем перекрывающиеся паттерны (оставляем первый по порядку в словаре)
        filtered_tokens = []
        used_positions = set()
        for start, end, token, pattern_name in special_tokens:
            # Проверяем, не перекрывается ли с уже использованными позициями
            overlap = False
            for pos in range(start, end):
                if pos in used_positions:
                    overlap = True
                    break
            if not overlap:
                filtered_tokens.append((start, end, token, pattern_name))
                used_positions.update(range(start, end))
        
        # Разбиваем текст на части
        last_pos = 0
        for start, end, token, pattern_name in filtered_tokens:
            # Добавляем обычные токены до специального
            if start > last_pos:
                regular_text = text[last_pos:start]
                regular_tokens = self._tokenize_regular(regular_text)
                tokens.extend(regular_tokens)
            # Добавляем специальный токен
            tokens.append(token)
            last_pos = end
        
        # Добавляем оставшиеся обычные токены
        if last_pos < len(text):
            regular_text = text[last_pos:]
            regular_tokens = self._tokenize_regular(regular_text)
            tokens.extend(regular_tokens)
        
        # Фильтруем пустые токены
        tokens = [t for t in tokens if t.strip()]
        
        return tokens
    
    def _tokenize_regular(self, text):
        """Токенизация обычного текста"""
        # Разбиваем по пробелам и знакам пунктуации, но сохраняем пунктуацию как отдельные токены
        # Обрабатываем апострофы в словах (don't, it's, etc.)
        tokens = []
        text = re.sub(r"(\w+)'(\w+)", r"\1'\2", text) 
        tokens = re.findall(r"\w+(?:'\w+)?|[^\w\s]", text)
        return tokens

tokenizer = AdvancedTokenizer()
print("Токенизатор создан")


Токенизатор создан


## Этап 3: Тестирование токенизации


In [3]:
# Тестирование на примерах
test_texts = [
    "Привет! Мой email: abc@example.com, телефон: +7-901-000-00-00",
    "Dr. Smith lives at г. Москва, ул. Ленина, д. 10, кв. 5",
    "Формула: a = b*c + (c+d)^2",
    "Стоимость: $100.50 или 50% скидка",
    "Привет :) как дела? ;-)"
]

for text in test_texts:
    print(f"\nТекст: {text}")
    sentences = tokenizer.segment_sentences(text)
    print(f"Предложения: {sentences}")
    for sent in sentences:
        tokens = tokenizer.tokenize(sent)
        print(f"Токены: {tokens}")



Текст: Привет! Мой email: abc@example.com, телефон: +7-901-000-00-00
Предложения: ['Привет!', 'Мой email: abc@example.com, телефон: +7-901-000-00-00']
Токены: ['Привет', '!']
Токены: ['Мой', 'email', ':', 'abc@example.com', ',', 'телефон', ':', '+7-901-000-00', '-', '00']

Текст: Dr. Smith lives at г. Москва, ул. Ленина, д. 10, кв. 5
Предложения: ['Dr. Smith lives at г. Москва, ул.', 'Ленина, д.', '10, кв.', '5']
Токены: ['Dr.', 'Smith', 'lives', 'at', 'г. Москва, ул.']
Токены: ['Ленина', ',', 'д', '.']
Токены: ['10', ',', 'кв', '.']
Токены: ['5']

Текст: Формула: a = b*c + (c+d)^2
Предложения: ['Формула: a = b*c + (c+d)^2']
Токены: ['Формула', ':', 'a = b*c + (c+d)^2']

Текст: Стоимость: $100.50 или 50% скидка
Предложения: ['Стоимость: $100.50 или 50% скидка']
Токены: ['Стоимость', ':', '$100.50', 'или', '50%', 'скидка']

Текст: Привет :) как дела? ;-)
Предложения: ['Привет :) как дела?', ';-)']
Токены: ['Привет', ':)', 'как', 'дела', '?']
Токены: [';-)']


## Этап 4: Стемминг и лемматизация


In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def get_stem(word):
    """Получить основу слова"""
    try:
        return stemmer.stem(word.lower())
    except:
        return word.lower()

def get_lemma(word):
    """Получить лемму слова"""
    try:
        for pos in ['n', 'v', 'a', 'r']:  # noun, verb, adjective, adverb
            lemma = lemmatizer.lemmatize(word.lower(), pos=pos)
            if lemma != word.lower():
                return lemma
        return lemmatizer.lemmatize(word.lower())
    except:
        return word.lower()

test_words = ['running', 'ran', 'better', 'best', 'cats', 'cat', 'went', 'goes', 'went']
print("Тестирование стемминга и лемматизации:")
for word in test_words:
    stem = get_stem(word)
    lemma = get_lemma(word)
    print(f"{word:10} -> stem: {stem:10} lemma: {lemma:10}")


Тестирование стемминга и лемматизации:
running    -> stem: run        lemma: run       
ran        -> stem: ran        lemma: run       
better     -> stem: better     lemma: good      
best       -> stem: best       lemma: best      
cats       -> stem: cat        lemma: cat       
cat        -> stem: cat        lemma: cat       
went       -> stem: went       lemma: go        
goes       -> stem: goe        lemma: go        
went       -> stem: went       lemma: go        


## Этап 5: Функция для создания аннотаций


In [5]:
def annotate_text(text):
    """Создает аннотацию текста в формате TSV"""
    annotations = []
    
    # Сегментируем на предложения
    sentences = tokenizer.segment_sentences(text)
    
    for sentence in sentences:
        # Токенизируем предложение
        tokens = tokenizer.tokenize(sentence)
        
        for token in tokens:
            # Проверяем, является ли токен специальным (email, phone, etc.)
            is_special = False
            for pattern_name, pattern in tokenizer.patterns.items():
                if re.match(pattern + '$', token, re.IGNORECASE):
                    is_special = True
                    break
            
            # Для специальных токенов и пунктуации используем сам токен как стем и лемму
            if is_special or (re.match(r'^[^\w\s]+$', token) and len(token) == 1):
                stem = token
                lemma = token
            elif re.match(r'^[^\w\s]+$', token):
                # Множественная пунктуация
                stem = token
                lemma = token
            else:
                # Получаем стем и лемму для обычных слов
                stem = get_stem(token)
                lemma = get_lemma(token)
            
            annotations.append({
                'token': token,
                'stem': stem,
                'lemma': lemma
            })
        
        # Добавляем пустую строку между предложениями
        annotations.append(None)
    
    return annotations

# Тестирование
test_text = "The cats are running. They went to the park."
annotations = annotate_text(test_text)
print("Пример аннотации:")
for ann in annotations:
    if ann is None:
        print()
    else:
        print(f"{ann['token']}\t{ann['stem']}\t{ann['lemma']}")


Пример аннотации:
The	the	the
cats	cat	cat
are	are	be
running	run	run
.	.	.

They	they	they
went	went	go
to	to	to
the	the	the
park	park	park
.	.	.



## Этап 6: Загрузка и обработка данных


In [ ]:
def load_csv_data(filepath):
    """Загружает данные из CSV файла"""
    data = []
    has_header = False
    doc_counter = 0
    
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        reader = csv.reader(f, quotechar='"', escapechar='\\')
        for idx, row in enumerate(reader):
            if idx == 0 and len(row) > 0 and row[0].lower() in ['doc_id', 'id', 'label', 'class']:
                has_header = True
                continue
            
            if len(row) >= 3:
                label = row[0].strip('"').strip()
                title = row[1].strip('"').strip() if len(row) > 1 else ""
                text = row[2].strip('"').strip() if len(row) > 2 else ""
                text = text.replace('\\n', '\n').replace('\\t', '\t').replace('\\\\', '\\')
                if title:
                    full_text = title + ' ' + text
                else:
                    full_text = text
                if len(row) > 3:
                    additional_text = ' '.join(row[3:]).strip('"').strip()
                    additional_text = additional_text.replace('\\n', '\n').replace('\\t', '\t').replace('\\\\', '\\')
                    full_text += ' ' + additional_text
                doc_id = str(doc_counter)
                doc_counter += 1
                data.append({
                    'doc_id': doc_id,
                    'label': label,
                    'text': full_text
                })
    return data

print("Загрузка train.csv...")
train_data = load_csv_data('train.csv')
print(f"Загружено {len(train_data)} документов из train.csv")

print("\nЗагрузка test.csv...")
test_data = load_csv_data('test.csv')
print(f"Загружено {len(test_data)} документов из test.csv")

print("\nПримеры из train.csv:")
for i, doc in enumerate(train_data[:3]):
    print(f"Doc {i+1}: ID={doc['doc_id']}, Label={doc['label']}, Text preview: {doc['text'][:100]}...")


Загрузка train.csv...
Загружено 119809 документов из train.csv

Загрузка test.csv...
Загружено 7590 документов из test.csv

Примеры из train.csv:
Doc 1: ID=0, Label=3, Text preview: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlingba...
Doc 2: ID=1, Label=3, Text preview: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,...
Doc 3: ID=2, Label=3, Text preview: Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worriesabout the...


## Этап 7: Сохранение аннотаций в TSV файлы


In [ ]:
def save_annotations_to_tsv(annotations, filepath):
    """Сохраняет аннотации в TSV файл"""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for ann in annotations:
            if ann is None:
                f.write('\n')
            else:
                token = ann['token'].replace('\t', ' ').replace('\n', ' ')
                stem = ann['stem'].replace('\t', ' ').replace('\n', ' ')
                lemma = ann['lemma'].replace('\t', ' ').replace('\n', ' ')
                f.write(f"{token}\t{stem}\t{lemma}\n")

def process_dataset(data, output_dir, dataset_name):
    """Обрабатывает датасет и сохраняет аннотации"""
    print(f"\nОбработка {dataset_name}...")
    
    errors = []
    for i, doc in enumerate(data):
        if i % 100 == 0:
            print(f"Обработано {i}/{len(data)} документов...")
        
        try:
            if not doc['text'] or not doc['text'].strip():
                continue
            
            annotations = annotate_text(doc['text'])
            
            label = doc['label'].strip()
            label = re.sub(r'[<>:"/\\|?*]', '_', label)
            if not label:
                label = 'unknown'
            
            label_dir = os.path.join(output_dir, label)
            os.makedirs(label_dir, exist_ok=True)
            
            doc_id = doc['doc_id'].strip()
            doc_id = re.sub(r'[<>:"/\\|?*]', '_', doc_id)
            if not doc_id:
                doc_id = 'unknown'
            
            filepath = os.path.join(label_dir, f"{doc_id}.tsv")
            
            save_annotations_to_tsv(annotations, filepath)
        except Exception as e:
            errors.append((doc['doc_id'], str(e)))
            print(f"Ошибка при обработке документа {doc['doc_id']}: {e}")
    
    print(f"Обработка {dataset_name} завершена!")
    if errors:
        print(f"Найдено {len(errors)} ошибок при обработке")

output_base = 'projects/LR1/assets/annotated-corpus'
train_output = os.path.join(output_base, 'train')
test_output = os.path.join(output_base, 'test')

print(f"Выходная директория: {output_base}")


Выходная директория: projects/LR1/assets/annotated-corpus


## Этап 8: Тестирование на небольшом примере перед полной обработкой


In [ ]:
print("Тестирование на первых 5 документах из train.csv:")
test_sample = train_data[:5]
test_output_sample = 'test_sample_output'

for doc in test_sample:
    print(f"\nДокумент ID: {doc['doc_id']}, Label: {doc['label']}")
    print(f"Текст (первые 200 символов): {doc['text'][:200]}...")
    
    annotations = annotate_text(doc['text'])
    
    print("Первые 10 аннотаций:")
    count = 0
    for ann in annotations:
        if ann is None:
            print()
            count += 1
            if count >= 2:
                break
        else:
            print(f"  {ann['token']}\t{ann['stem']}\t{ann['lemma']}")
            count += 1
            if count >= 10:
                break


Тестирование на первых 5 документах из train.csv:

Документ ID: 0, Label: 3
Текст (первые 200 символов): Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlingband of ultra-cynics, are seeing green again....
Первые 10 аннотаций:
  Wall	wall	wall
  St.	St.	St.
  Bears	bear	bear
  Claw	claw	claw
  Back	back	back
  Into	into	into
  the	the	the
  Black	black	black
  (	(	(
  Reuters	reuter	reuters

Документ ID: 1, Label: 3
Текст (первые 200 символов): Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,which has a reputation for making well-timed and occasionallycontroversial plays in the defense indu...
Первые 10 аннотаций:
  Carlyle	carlyl	carlyle
  Looks	look	look
  Toward	toward	toward
  Commercial	commerci	commercial
  Aerospace	aerospac	aerospace
  (	(	(
  Reuters	reuter	reuters
  )	)	)
  Reuters	reuter	reuters
  -	-	-

Документ ID: 2, Label: 3
Текст (первые 200 символов): Oil and Economy Cloud

## Этап 9: Обработка train и test датасетов


In [ ]:
process_dataset(train_data, train_output, 'train')



Обработка train...
Обработано 0/119809 документов...
Обработано 100/119809 документов...
Обработано 200/119809 документов...
Обработано 300/119809 документов...
Обработано 400/119809 документов...
Обработано 500/119809 документов...
Обработано 600/119809 документов...
Обработано 700/119809 документов...
Обработано 800/119809 документов...
Обработано 900/119809 документов...
Обработано 1000/119809 документов...
Обработано 1100/119809 документов...
Обработано 1200/119809 документов...
Обработано 1300/119809 документов...
Обработано 1400/119809 документов...
Обработано 1500/119809 документов...
Обработано 1600/119809 документов...
Обработано 1700/119809 документов...
Обработано 1800/119809 документов...
Обработано 1900/119809 документов...
Обработано 2000/119809 документов...
Обработано 2100/119809 документов...
Обработано 2200/119809 документов...
Обработано 2300/119809 документов...
Обработано 2400/119809 документов...
Обработано 2500/119809 документов...
Обработано 2600/119809 докумен

In [ ]:
process_dataset(test_data, test_output, 'test')



Обработка test...
Обработано 0/7590 документов...
Обработано 100/7590 документов...
Обработано 200/7590 документов...
Обработано 300/7590 документов...
Обработано 400/7590 документов...
Обработано 500/7590 документов...
Обработано 600/7590 документов...
Обработано 700/7590 документов...
Обработано 800/7590 документов...
Обработано 900/7590 документов...
Обработано 1000/7590 документов...
Обработано 1100/7590 документов...
Обработано 1200/7590 документов...
Обработано 1300/7590 документов...
Обработано 1400/7590 документов...
Обработано 1500/7590 документов...
Обработано 1600/7590 документов...
Обработано 1700/7590 документов...
Обработано 1800/7590 документов...
Обработано 1900/7590 документов...
Обработано 2000/7590 документов...
Обработано 2100/7590 документов...
Обработано 2200/7590 документов...
Обработано 2300/7590 документов...
Обработано 2400/7590 документов...
Обработано 2500/7590 документов...
Обработано 2600/7590 документов...
Обработано 2700/7590 документов...
Обработано 28

In [ ]:
process_dataset(test_data, test_output, 'test')



Обработка test...
Обработано 0/7590 документов...
Обработано 100/7590 документов...
Обработано 200/7590 документов...
Обработано 300/7590 документов...
Обработано 400/7590 документов...
Обработано 500/7590 документов...
Обработано 600/7590 документов...
Обработано 700/7590 документов...
Обработано 800/7590 документов...
Обработано 900/7590 документов...
Обработано 1000/7590 документов...
Обработано 1100/7590 документов...
Обработано 1200/7590 документов...
Обработано 1300/7590 документов...
Обработано 1400/7590 документов...
Обработано 1500/7590 документов...
Обработано 1600/7590 документов...
Обработано 1700/7590 документов...
Обработано 1800/7590 документов...
Обработано 1900/7590 документов...
Обработано 2000/7590 документов...
Обработано 2100/7590 документов...
Обработано 2200/7590 документов...
Обработано 2300/7590 документов...
Обработано 2400/7590 документов...
Обработано 2500/7590 документов...
Обработано 2600/7590 документов...
Обработано 2700/7590 документов...
Обработано 28

## Этап 9: Проверка результатов и анализ омонимии


In [ ]:
import glob

train_files = glob.glob(os.path.join(train_output, '**', '*.tsv'), recursive=True)
test_files = glob.glob(os.path.join(test_output, '**', '*.tsv'), recursive=True)

print(f"Создано файлов в train: {len(train_files)}")
print(f"Создано файлов в test: {len(test_files)}")

if train_files:
    print(f"\nПример содержимого файла {train_files[0]}:")
    with open(train_files[0], 'r', encoding='utf-8') as f:
        lines = f.readlines()[:10]
        for line in lines:
            print(line.rstrip())


Создано файлов в train: 119809
Создано файлов в test: 7590

Пример содержимого файла projects/LR1/assets/annotated-corpus\train\1\1000.tsv:
Fischer	fischer	fischer
Appeals	appeal	appeal
to	to	to
Powell	powel	powell
to	to	to
Help	help	help
Him	him	him
Renounce	renounc	renounce
U	u	u
.	.	.


In [ ]:
# Анализ омонимии - находим слова, для которых лемматизатор дает разные результаты
# в зависимости от контекста

def analyze_homonymy(text):
    """Анализирует случаи омонимии в тексте"""
    homonyms = {}
    sentences = tokenizer.segment_sentences(text)
    
    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        for token in tokens:
            if re.match(r'^[a-zA-Z]+$', token):  # Только слова
                # Пробуем разные части речи
                lemmas = {}
                for pos in ['n', 'v', 'a', 'r']:
                    lemma = lemmatizer.lemmatize(token.lower(), pos=pos)
                    if lemma != token.lower():
                        lemmas[pos] = lemma
                
                if len(lemmas) > 1:
                    if token.lower() not in homonyms:
                        homonyms[token.lower()] = lemmas
    
    return homonyms

test_text = "The bank is near the river bank. I can can the tomatoes."
homonyms = analyze_homonymy(test_text)
print("Примеры омонимии:")
for word, lemmas in homonyms.items():
    print(f"  {word}: {lemmas}")

print("\nПоиск омонимов в датасете...")
sample_texts = [doc['text'] for doc in train_data[:100]]
all_homonyms = {}
for text in sample_texts:
    h = analyze_homonymy(text)
    all_homonyms.update(h)

print(f"\nНайдено {len(all_homonyms)} слов с потенциальной омонимией:")
for word, lemmas in list(all_homonyms.items())[:10]:
    print(f"  {word}: {lemmas}")


Примеры омонимии:

Поиск омонимов в датасете...

Найдено 134 слов с потенциальной омонимией:
  bears: {'n': 'bear', 'v': 'bear'}
  looks: {'n': 'look', 'v': 'look'}
  has: {'n': 'ha', 'v': 'have'}
  plays: {'n': 'play', 'v': 'play'}
  bets: {'n': 'bet', 'v': 'bet'}
  stocks: {'n': 'stock', 'v': 'stock'}
  prices: {'n': 'price', 'v': 'price'}
  halts: {'n': 'halt', 'v': 'halt'}
  exports: {'n': 'export', 'v': 'export'}
  records: {'n': 'record', 'v': 'record'}
